## Библиотеки

In [1]:
import pandas as pd
from scipy.stats import mannwhitneyu, ttest_ind, \
                        chi2_contingency, shapiro


## Задание

### Знакомство с данными

In [2]:
# История заказов пользователей
ab_users_data = pd.read_csv('C:/Users/Arthur/Desktop/ab_users_data.csv',
                            parse_dates=['time', 'date'])

# Состав заказов
ab_orders = pd.read_csv('C:/Users/Arthur/Desktop/ab_orders.csv',
                        parse_dates=['creation_time'])

# Информация о продуктах
ab_products = pd.read_csv('C:/Users/Arthur/Desktop/ab_products.csv')

In [3]:
ab_users_data.head()

,user_id,order_id,action,time,date,group
0,964,1255,create_order,2022-08-26 00:00:19,2022-08-26,0
1,965,1256,create_order,2022-08-26 00:02:21,2022-08-26,1
2,964,1257,create_order,2022-08-26 00:02:27,2022-08-26,0
3,966,1258,create_order,2022-08-26 00:02:56,2022-08-26,0
4,967,1259,create_order,2022-08-26 00:03:37,2022-08-26,1


In [4]:
ab_orders.head()

,order_id,creation_time,product_ids
0,1255,2022-08-26 00:00:19,"{75, 22, 53, 84}"
1,1256,2022-08-26 00:02:21,"{56, 76, 39}"
2,1257,2022-08-26 00:02:27,"{76, 34, 41, 38}"
3,1258,2022-08-26 00:02:56,"{74, 6}"
4,1259,2022-08-26 00:03:37,"{20, 45, 67, 26}"


In [5]:
ab_products.head()

,product_id,name,price
0,1,сахар,150.0
1,2,чай зеленый в пакетиках,50.0
2,3,вода негазированная,80.4
3,4,леденцы,45.5
4,5,кофе 3 в 1,15.0


Для начала к таблице с историей заказов присоединим информацию о составах заказов

In [6]:
ab = ab_users_data.merge(ab_orders, on='order_id')

In [7]:
ab.head()

,user_id,order_id,action,time,date,group,creation_time,product_ids
0,964,1255,create_order,2022-08-26 00:00:19.000000,2022-08-26,0,2022-08-26 00:00:19,"{75, 22, 53, 84}"
1,965,1256,create_order,2022-08-26 00:02:21.000000,2022-08-26,1,2022-08-26 00:02:21,"{56, 76, 39}"
2,964,1257,create_order,2022-08-26 00:02:27.000000,2022-08-26,0,2022-08-26 00:02:27,"{76, 34, 41, 38}"
3,966,1258,create_order,2022-08-26 00:02:56.000000,2022-08-26,0,2022-08-26 00:02:56,"{74, 6}"
4,966,1258,cancel_order,2022-08-26 00:08:25.486419,2022-08-26,0,2022-08-26 00:02:56,"{74, 6}"


Приведём колонки time и creation_time к одному формату. Также поменяем  
порядок столбцов таблицы.

In [8]:
ab.time = ab.time.dt.strftime('%Y-%m-%d %H:%M:%S')

In [9]:
ab = ab.reindex(columns=['user_id', 'order_id', 'group', 'action',
                         'date', 'creation_time', 'time', 'product_ids'])

Для начала взглянем на получившуюся таблицу. Посмотрим её размер,  
число уникальных значений.

In [10]:
ab.head()

,user_id,order_id,group,action,date,creation_time,time,product_ids
0,964,1255,0,create_order,2022-08-26,2022-08-26 00:00:19,2022-08-26 00:00:19,"{75, 22, 53, 84}"
1,965,1256,1,create_order,2022-08-26,2022-08-26 00:02:21,2022-08-26 00:02:21,"{56, 76, 39}"
2,964,1257,0,create_order,2022-08-26,2022-08-26 00:02:27,2022-08-26 00:02:27,"{76, 34, 41, 38}"
3,966,1258,0,create_order,2022-08-26,2022-08-26 00:02:56,2022-08-26 00:02:56,"{74, 6}"
4,966,1258,0,cancel_order,2022-08-26,2022-08-26 00:02:56,2022-08-26 00:08:25,"{74, 6}"


In [11]:
ab.shape

(4337, 8)

In [12]:
ab.nunique()

user_id          1017
order_id         4123
group               2
action              2
date               14
creation_time    4098
time             4310
product_ids      3877
dtype: int64

### Сравнение числа отмен

В столбце action некоторые из заказов были отменены. Создадим таблицу  
сопряжённости (где строки это группы, а столбцы результат операции  
(покупка/отмена)) и проверим, существует ли статистически значимые  
различия между группами.

In [13]:
canceled_orders = ab[ab.action == 'cancel_order'].groupby('group') \
                                                 .count() \
                                                 .action \
                                                 .rename('canceled')

In [14]:
canceled_orders

group
0     82
1    132
Name: canceled, dtype: int64

In [15]:
success_orders = ab[ab.action == 'create_order'].groupby('group') \
                                                .count() \
                                                .action

# Вычитаем отмененные заказы, т.к. отмененный заказ также имеет статус создания
success_orders = (success_orders - canceled_orders).rename('success')

In [16]:
success_orders

group
0    1527
1    2382
Name: success, dtype: int64

Найдём процент отмен для каждой группы:

In [17]:
canceled_orders.div(success_orders).mul(100).round(2)

group
0    5.37
1    5.54
dtype: float64

Составим таблицу сопряжённости и проверим статистическую значимость  
процента отмен.

In [18]:
canceled_success_crosstab = pd.concat([canceled_orders, success_orders], axis=1)

In [19]:
canceled_success_crosstab

,canceled,success
group,,
0,82,1527
1,132,2382


Так как мы имеем дело с категориальными переменными и таблицей 2x2,  
со значением ожидаемых и наблюдаемых частот выше 10, применим  
критерий хи-квадрат.

Сформулируем гипотезы:

* H<sub>0</sub>: Взаимосвязи между алгоритмом и числом отмен нет;
* H<sub>1</sub>: Взаимосвязь есть.

In [20]:
p = chi2_contingency(canceled_success_crosstab)[1]
print(f'p-value = {p}')

alpha = 0.05
if p <= alpha:
    print('Отклоняем H0. Взаимосвязь доказана.')
else:
    print('Не отклоняем H0. Взимосвязи НЕ доказана.')

p-value = 0.8840344321879333
Не отклоняем H0. Взимосвязи НЕ доказана.


### Сравнение активности пользователей

Для проверки следующий гипотез информация только об успешних покупках.

In [21]:
ab.head()

,user_id,order_id,group,action,date,creation_time,time,product_ids
0,964,1255,0,create_order,2022-08-26,2022-08-26 00:00:19,2022-08-26 00:00:19,"{75, 22, 53, 84}"
1,965,1256,1,create_order,2022-08-26,2022-08-26 00:02:21,2022-08-26 00:02:21,"{56, 76, 39}"
2,964,1257,0,create_order,2022-08-26,2022-08-26 00:02:27,2022-08-26 00:02:27,"{76, 34, 41, 38}"
3,966,1258,0,create_order,2022-08-26,2022-08-26 00:02:56,2022-08-26 00:02:56,"{74, 6}"
4,966,1258,0,cancel_order,2022-08-26,2022-08-26 00:02:56,2022-08-26 00:08:25,"{74, 6}"


In [22]:
canceled = ab[ab.action == 'cancel_order'].order_id

In [23]:
ab_purchases = ab.query('order_id not in @canceled').copy()

In [24]:
ab_purchases.head()

,user_id,order_id,group,action,date,creation_time,time,product_ids
0,964,1255,0,create_order,2022-08-26,2022-08-26 00:00:19,2022-08-26 00:00:19,"{75, 22, 53, 84}"
1,965,1256,1,create_order,2022-08-26,2022-08-26 00:02:21,2022-08-26 00:02:21,"{56, 76, 39}"
2,964,1257,0,create_order,2022-08-26,2022-08-26 00:02:27,2022-08-26 00:02:27,"{76, 34, 41, 38}"
5,967,1259,1,create_order,2022-08-26,2022-08-26 00:03:37,2022-08-26 00:03:37,"{20, 45, 67, 26}"
6,968,1261,0,create_order,2022-08-26,2022-08-26 00:05:35,2022-08-26 00:05:35,"{30, 35, 69, 6}"


Найдём число пользователей старого и нового алгоритма.

In [25]:
# Старый алгоритм
ab_purchases[ab_purchases.group == 0].user_id.nunique()

512

In [26]:
# Новый алгоритм
ab_purchases[ab_purchases.group == 1].user_id.nunique()

501

Число пользователей со старым и новым алгоритмом примерно одинаковы,  
проверим, есть ли разница в их активности: найдём ежедневное число  
действий пользователей в каждой из групп.

In [27]:
date_group_crosstab = pd.crosstab(ab_purchases.date, ab_purchases.group)

In [28]:
date_group_crosstab

group,0,1
date,,
2022-08-26,595,655
2022-08-27,105,186
2022-08-28,89,200
2022-08-29,102,216
2022-08-30,96,151
2022-08-31,64,119
2022-09-01,70,126
2022-09-02,72,134
2022-09-03,62,140


Невооружённом взглядом видно, что пользователи в тестовой группе совершили  
большее число покупок. Тем не менее проверим среднее ежедневное число заказов.

In [29]:
date_group_crosstab.mean().round(2)

group
0    109.07
1    170.14
dtype: float64

Проверим нормальность распределения и найдём статистическую значимость  
получившихся различий.

In [30]:
p = shapiro(date_group_crosstab[0])[1]
print(f'p-value = {p}')

alpha = 0.05
if p <= alpha:
    print('Ненормальное распределение')
else:
    print('Нормальное распределение')

p-value = 1.9268536561867222e-06
Ненормальное распределение


In [31]:
p = shapiro(date_group_crosstab[1])[1]
print(f'p-value = {p}')

alpha = 0.05
if p <= alpha:
    print('Ненормальное распределение')
else:
    print('Нормальное распределение')

p-value = 3.6935518437530845e-05
Ненормальное распределение


Вычислим статистическую значимость различий. Для проверки  
используем U-критерий Манна-Уитни, т.к. распределение  
не является нормальным.

Сформулируем гипотезы:

* H<sub>0</sub>: Взаимосвязи между алгоритмом и числом покупок нет;
* H<sub>1</sub>: Взаимосвязь есть.

In [32]:
p = mannwhitneyu(x=date_group_crosstab[0], y=date_group_crosstab[1])[1]
print(f'p-value = {p}')

alpha = 0.05
if p <= alpha:
    print('Отклоняем H0. Взаимосвязь доказана.')
else:
    print('Не отклоняем H0. Взимосвязи НЕ доказана.')

p-value = 0.00352648665061584
Отклоняем H0. Взаимосвязь доказана.


### Сравнение числа товаров в чеке

Далее мы сравним среднее число товаров в корзине. Для следующих  
сравнений необходимо "разбить" product_ids (список покупок). Также  
присоединим к датафрейму информацию с ценами для следующего теста.

In [33]:
ab_purchases.head()

,user_id,order_id,group,action,date,creation_time,time,product_ids
0,964,1255,0,create_order,2022-08-26,2022-08-26 00:00:19,2022-08-26 00:00:19,"{75, 22, 53, 84}"
1,965,1256,1,create_order,2022-08-26,2022-08-26 00:02:21,2022-08-26 00:02:21,"{56, 76, 39}"
2,964,1257,0,create_order,2022-08-26,2022-08-26 00:02:27,2022-08-26 00:02:27,"{76, 34, 41, 38}"
5,967,1259,1,create_order,2022-08-26,2022-08-26 00:03:37,2022-08-26 00:03:37,"{20, 45, 67, 26}"
6,968,1261,0,create_order,2022-08-26,2022-08-26 00:05:35,2022-08-26 00:05:35,"{30, 35, 69, 6}"


In [34]:
# Превращаем product_ids в список
ab_purchases.product_ids = ab_purchases.product_ids.str.strip('{}').str.split(', ')

In [35]:
# Разбиваем product_ids, а также переименовываем для join'а
ab_explode = ab_purchases.explode('product_ids').rename(columns={'product_ids' : 'product_id'})

In [36]:
# Меняем формат колонки для join'а
ab_explode.product_id = ab_explode.product_id.astype('int64')

In [37]:
ab_explode.head()

,user_id,order_id,group,action,date,creation_time,time,product_id
0,964,1255,0,create_order,2022-08-26,2022-08-26 00:00:19,2022-08-26 00:00:19,75
0,964,1255,0,create_order,2022-08-26,2022-08-26 00:00:19,2022-08-26 00:00:19,22
0,964,1255,0,create_order,2022-08-26,2022-08-26 00:00:19,2022-08-26 00:00:19,53
0,964,1255,0,create_order,2022-08-26,2022-08-26 00:00:19,2022-08-26 00:00:19,84
1,965,1256,1,create_order,2022-08-26,2022-08-26 00:02:21,2022-08-26 00:02:21,56


In [38]:
ab_products.head()

,product_id,name,price
0,1,сахар,150.0
1,2,чай зеленый в пакетиках,50.0
2,3,вода негазированная,80.4
3,4,леденцы,45.5
4,5,кофе 3 в 1,15.0


In [39]:
ab_full = ab_explode.merge(ab_products, on='product_id')

In [40]:
ab_full.head()

,user_id,order_id,group,action,date,creation_time,time,product_id,name,price
0,964,1255,0,create_order,2022-08-26,2022-08-26 00:00:19,2022-08-26 00:00:19,75,сок ананасовый,120.0
1,987,1287,0,create_order,2022-08-26,2022-08-26 00:31:36,2022-08-26 00:31:36,75,сок ананасовый,120.0
2,1073,1403,1,create_order,2022-08-26,2022-08-26 03:01:40,2022-08-26 03:01:40,75,сок ананасовый,120.0
3,1089,1424,1,create_order,2022-08-26,2022-08-26 04:01:22,2022-08-26 04:01:22,75,сок ананасовый,120.0
4,1139,1495,1,create_order,2022-08-26,2022-08-26 06:04:05,2022-08-26 06:04:05,75,сок ананасовый,120.0


Отсортируем таблицу по номеру заказа и номеру продукта.

In [41]:
ab_full = ab_full.sort_values(['order_id', 'product_id'])

In [42]:
ab_full.head()

,user_id,order_id,group,action,date,creation_time,time,product_id,name,price
112,964,1255,0,create_order,2022-08-26,2022-08-26 00:00:19,2022-08-26 00:00:19,22,сок мультифрукт,120.0
192,964,1255,0,create_order,2022-08-26,2022-08-26 00:00:19,2022-08-26 00:00:19,53,мука,78.3
0,964,1255,0,create_order,2022-08-26,2022-08-26 00:00:19,2022-08-26 00:00:19,75,сок ананасовый,120.0
354,964,1255,0,create_order,2022-08-26,2022-08-26 00:00:19,2022-08-26 00:00:19,84,мандарины,90.4
1041,965,1256,1,create_order,2022-08-26,2022-08-26 00:02:21,2022-08-26 00:02:21,39,бублики,45.0


Посчитаем число товаров в каждом заказе и найдём среднее число товаров  
в корзине для каждой группы пользователей.

In [43]:
control_purchases = ab_full[ab_full.group == 0].groupby('order_id', as_index=False) \
                                               .count() \
                                               .name

In [44]:
control_purchases.mean().round(2)

3.34

In [45]:
test_purchases = ab_full[ab_full.group == 1].groupby('order_id', as_index=False) \
                                               .count() \
                                               .name

In [46]:
test_purchases.mean().round(2)

3.35

Проверем нормальность распределения числа товаров в чеке со старым  
и с новым алгоритмом.

In [47]:
p = shapiro(control_purchases.value_counts())[1]
print(f'p-value = {p}')

alpha = 0.05
if p <= alpha:
    print('Ненормальное распределение')
else:
    print('Нормальное распределение')

p-value = 0.3445640504360199
Нормальное распределение


In [48]:
p = shapiro(test_purchases.value_counts())[1]
print(f'p-value = {p}')

alpha = 0.05
if p <= alpha:
    print('Ненормальное распределение')
else:
    print('Нормальное распределение')

p-value = 0.2954893112182617
Нормальное распределение


Распределения являются нормальными —  формулируем гипотезы и проводим t-тест.

Сформулируем гипотезы:

* H<sub>0</sub>: Взаимосвязи между алгоритмом и числом товаров в корзине нет;
* H<sub>1</sub>: Взаимосвязь есть.

In [49]:
p = ttest_ind(test_purchases, control_purchases)[1]
print(f'p-value = {p}')

alpha = 0.05
if p <= alpha:
    print('Отклоняем H0. Взаимосвязь доказана.')
else:
    print('Не отклоняем H0. Взимосвязи НЕ доказана.')

p-value = 0.84432251088501
Не отклоняем H0. Взимосвязи НЕ доказана.


### Сравнение суммы чека

Подсчитаем сумму каждой покупки для пользователей из обеих групп.

In [50]:
control_total = ab_full[ab_full.group == 0].groupby('order_id', as_index=False) \
                         .agg({'price' : 'sum'}) \
                         .rename(columns={'price' : 'total'})

In [51]:
control_total.head()

,order_id,total
0,1255,408.7
1,1257,310.2
2,1261,430.7
3,1262,358.6
4,1265,546.4


In [52]:
test_total = ab_full[ab_full.group == 1].groupby('order_id', as_index=False) \
                         .agg({'price' : 'sum'}) \
                         .rename(columns={'price' : 'total'})

In [53]:
test_total.head()

,order_id,total
0,1256,250.5
1,1259,228.0
2,1263,180.3
3,1264,310.3
4,1266,46.0


Найдём средний чек для тестовой и контрольной группы.

In [54]:
test_total.total.mean().round(2)

368.86

In [55]:
control_total.total.mean().round(2)

382.09

Проверим данные на нормальность распределения.

In [56]:
p = shapiro(control_total.total)[1]
print(f'p-value = {p}')

alpha = 0.05
if p <= alpha:
    print('Ненормальное распределение')
else:
    print('Нормальное распределение')

p-value = 3.810261968646895e-28
Ненормальное распределение


In [57]:
p = shapiro(test_total.total)[1]
print(f'p-value = {p}')

alpha = 0.05
if p <= alpha:
    print('Ненормальное распределение')
else:
    print('Нормальное распределение')

p-value = 8.38740133277359e-37
Ненормальное распределение


Распределения ненормальные. Формулируем гипотезы  
и применяем U-тест Манна-Уитни т.к. распределение  
не является нормальным.

Сформулируем гипотезы:

* H<sub>0</sub>: Взаимосвязи между алгоритмом и суммой чека нет;
* H<sub>1</sub>: Взаимосвязь есть.

In [58]:
p = mannwhitneyu(control_total.total, test_total.total)[1]
print(f'p-value = {p}')

alpha = 0.05
if p <= alpha:
    print('Отклоняем H0. Взаимосвязь доказана.')
else:
    print('Не отклоняем H0. Взимосвязи НЕ доказана.')

p-value = 0.07538521327564442
Не отклоняем H0. Взимосвязи НЕ доказана.


### Сравнение выручки

На данный момент мы не смогли обнаружить значимые различия:

* числа отмен;
* числа товаров в корзине;
* суммы чека.

Но обнаружили значимый рост числа покупок в тестовой группе.

Проведём ещё один тест — сравним выручку тестовой и контрольной группы.  
Создадим таблицу с датой и выручкой в каждой группе.

In [59]:
control_total_by_date = ab_full[ab_full.group == 0].groupby('date', as_index=False) \
                         .agg({'price' : 'sum'}) \
                         .rename(columns={'price' : 'control'})

In [60]:
control_total_by_date

,date,control
0,2022-08-26,228375.1
1,2022-08-27,39458.8
2,2022-08-28,31033.2
3,2022-08-29,41496.3
4,2022-08-30,34511.5
5,2022-08-31,26201.5
6,2022-09-01,27849.1
7,2022-09-02,31535.7
8,2022-09-03,25395.5
9,2022-09-04,29310.8


In [61]:
test_total_by_date = ab_full[ab_full.group == 1].groupby('date', as_index=False) \
                         .agg({'price' : 'sum'}) \
                         .rename(columns={'price' : 'test'})

In [62]:
test_total_by_date

,date,test
0,2022-08-26,235186.6
1,2022-08-27,70903.0
2,2022-08-28,75167.7
3,2022-08-29,83459.4
4,2022-08-30,52759.4
5,2022-08-31,43988.8
6,2022-09-01,46049.2
7,2022-09-02,50194.0
8,2022-09-03,56291.6
9,2022-09-04,51891.2


In [63]:
total_by_date = control_total_by_date.merge(test_total_by_date, on='date')

In [64]:
total_by_date

,date,control,test
0,2022-08-26,228375.1,235186.6
1,2022-08-27,39458.8,70903.0
2,2022-08-28,31033.2,75167.7
3,2022-08-29,41496.3,83459.4
4,2022-08-30,34511.5,52759.4
5,2022-08-31,26201.5,43988.8
6,2022-09-01,27849.1,46049.2
7,2022-09-02,31535.7,50194.0
8,2022-09-03,25395.5,56291.6
9,2022-09-04,29310.8,51891.2


Средняя ежедневная выручка по группам.

In [65]:
total_by_date[['control', 'test']].mean().round(2)

control    41675.42
test       62759.17
dtype: float64

Как и в случае с числом покупок, у тестовой группы более  
лучшие показатели. Докажем их статистическую значимость.  
Проверим нормальность распределение выручки для каждой группы.

In [66]:
p = shapiro(total_by_date.control)[1]
print(f'p-value = {p}')

alpha = 0.05
if p <= alpha:
    print('Ненормальное распределение')
else:
    print('Нормальное распределение')

p-value = 2.0419224711076822e-06
Ненормальное распределение


In [67]:
p = shapiro(total_by_date.test)[1]
print(f'p-value = {p}')

alpha = 0.05
if p <= alpha:
    print('Ненормальное распределение')
else:
    print('Нормальное распределение')

p-value = 7.645706500625238e-05
Ненормальное распределение


Вычислим статистическую значимость различий. Для проверки  
используем U-критерий Манна-Уитни, т.к. распределение  
не является нормальным.

Сформулируем гипотезы:

* H<sub>0</sub>: Взаимосвязи между алгоритмом и суммой выручки нет;
* H<sub>1</sub>: Взаимосвязь есть.

In [68]:
p = mannwhitneyu(total_by_date.control, total_by_date.test)[1]
print(f'p-value = {p}')

alpha = 0.05
if p <= alpha:
    print('Отклоняем H0. Взаимосвязь доказана.')
else:
    print('Не отклоняем H0. Взимосвязи НЕ доказана.')

p-value = 0.004716530360235347
Отклоняем H0. Взаимосвязь доказана.


### Вывод

По итогам 5 статистических тестов мы выяснили что между старым  
и новым алгоритмом не обнаружены статистические различия в:

* числе отмен заказов;
* числе товаров в корзине;
* сумме чека.

Однако следующие показатели значимо выросли в тестовой группе:

* ежедневное число количестве покупок;
* ежедневная выручка.

Включаем новую рекомендательную систему для всех пользователей.